# eQTL catalogue with HDF5 and python (pandas and PyTables implementation)

The python library [PyTables](https://www.pytables.org/index.html) is a python binding to the HDF library. HDF5 is a file format allowing one to access subsets of huge volumes of data very quickly. [Here](https://github.com/EBISPOT/SumStats/tree/eqtls) we have tools that convert .csv to .h5, index those files on specified fields, enabling the files accessed via complex queries. This notebook should show the basics, but also see the repo above where one can pip install some utilites to help access the files (_still in construction_).

Requires:
- `python=3.6`
- `pandas=0.19.2`
- `pytables=3.4.3`
- `hdf5=1.10.2`

Easiest way to do this is:
- Clone the repository
  - `git clone https://github.com/EBISPOT/SumStats.git`
  - `cd SumStats`
- Create conda environment (installs HDF5 lib and other dependencies)
	- `conda env create -f sumstats.yml`
	- `conda  activate  sumstats`
- Run everything from within that environment
- When you're finished, simply deactivate
    - `conda deactivate`

### Examples

##### 1) Let's take a look at a file using a great pytables utility [`ptdump`](https://www.pytables.org/usersguide/utilities.html#ptdump)

In [10]:
%%bash
hdf=files/output/bystudy/1/file_LCL.nominal.sorted.head.h5 # path to your hdf
ptdump -v $hdf

/ (RootGroup) ''
/GEUVADIS (Group) ''
/GEUVADIS/table (Table(1000,)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "values_block_0": Int64Col(shape=(2,), dflt=0, pos=1),
  "values_block_1": Float64Col(shape=(4,), dflt=0.0, pos=2),
  "values_block_2": StringCol(itemsize=5, shape=(1,), dflt=b'', pos=3),
  "molecular_trait_id": StringCol(itemsize=255, shape=(), dflt=b'', pos=4),
  "chromosome": StringCol(itemsize=1, shape=(), dflt=b'', pos=5),
  "position": Int64Col(shape=(), dflt=0, pos=6),
  "pvalue": Float64Col(shape=(), dflt=0.0, pos=7),
  "ref": StringCol(itemsize=255, shape=(), dflt=b'', pos=8),
  "alt": StringCol(itemsize=255, shape=(), dflt=b'', pos=9),
  "gene_id": StringCol(itemsize=255, shape=(), dflt=b'', pos=10),
  "molecular_trait_object_id": StringCol(itemsize=255, shape=(), dflt=b'', pos=11),
  "variant": StringCol(itemsize=255, shape=(), dflt=b'', pos=12)}
  byteorder := 'little'
  chunkshape := (81,)
  autoindex := True
  colindexes := {
    "mole

The output from the above, shows us an output much like the HDF lib utility `h5dump`. It shows us the fields and of particular importance the `colindexes`, which we are the fields we can use to form complex queries.

##### 2) Read all data into a pandas dataframe using [pandas.HDFStore.select()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.HDFStore.select.html)

In [8]:
import pandas as pd

hdf = "files/output/bystudy/1/file_LCL.nominal.sorted.head.h5" # path to hdf

with pd.HDFStore(hdf, mode='r') as store:
    key = store.keys()[0]
    df = store.select(key)
    
    #Do what you like with the dataframe
    
    print(df.head())

     pvalue   an   gene_id  r2  position alt      beta  \
0  0.398658  716  clu_2529 NaN     10177  AC -0.078357   
1  0.953312  716  clu_2529 NaN     10177  AC -0.005483   
2  0.891248  716  clu_2525 NaN     10177  AC  0.012683   
3  0.505367  716  clu_2525 NaN     10177  AC  0.061232   
4  0.527385  716  clu_2531 NaN     10177  AC  0.059108   

         molecular_trait_id          variant  median_tpm  \
0  1:630476:634113:clu_2529  chr1_10177_A_AC         NaN   
1  1:632019:634113:clu_2529  chr1_10177_A_AC         NaN   
2  1:729955:732013:clu_2525  chr1_10177_A_AC         NaN   
3  1:729955:732017:clu_2525  chr1_10177_A_AC         NaN   
4  1:739024:739803:clu_2531  chr1_10177_A_AC         NaN   

  molecular_trait_object_id       maf chromosome   ac   type ref  
0                       NaN  0.406425          1  291  INDEL   A  
1                       NaN  0.406425          1  291  INDEL   A  
2                       NaN  0.406425          1  291  INDEL   A  
3                     

##### 3) Let's say we're only interested in rows with pvalues < 0.01

In [13]:
import pandas as pd

hdf = "files/output/bystudy/1/file_LCL.nominal.sorted.head.h5" # path to hdf

query = "pvalue < 0.01"

with pd.HDFStore(hdf, mode='r') as store:
    key = store.keys()[0]
    df = store.select(key, where=query)
    
    #Do what you like with the dataframe
    
    print(df.head())

       pvalue   an   gene_id  r2  position alt      beta  \
364  0.006946  716  clu_2538 NaN     13110   A  0.452185   
387  0.004229  716  clu_2543 NaN     13110   A -0.464828   
388  0.007822  716  clu_2543 NaN     13110   A  0.435086   
607  0.007159  716  clu_2538 NaN     13273   C -0.271604   
674  0.004420  716  clu_2534 NaN     14464   T  0.286950   

           molecular_trait_id         variant  median_tpm  \
364  1:844498:847654:clu_2538  chr1_13110_G_A         NaN   
387  1:954082:955923:clu_2543  chr1_13110_G_A         NaN   
388  1:954523:955923:clu_2543  chr1_13110_G_A         NaN   
607  1:829104:851927:clu_2538  chr1_13273_G_C         NaN   
674  1:774280:778284:clu_2534  chr1_14464_A_T         NaN   

           molecular_trait_object_id       maf chromosome   ac type ref  
364                  ENSG00000228794  0.053073          1   38  SNP   G  
387                  ENSG00000188976  0.053073          1   38  SNP   G  
388                  ENSG00000188976  0.053073    

##### 4) We can make complex queries by concatenating conditions with '&' e.g. only rows with pvalues < 0.01 and position < 14000 (see more [here](http://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#querying)): 

In [21]:
import pandas as pd

hdf = "files/output/bystudy/1/file_LCL.nominal.sorted.head.h5" # path to hdf

query = "pvalue < 0.01 & position < 14000"

with pd.HDFStore(hdf, mode='r') as store:
    key = store.keys()[0]
    df = store.select(key, where=query)
    
    #Do what you like with the dataframe
    
    print(df.head())

       pvalue   an   gene_id  r2  position alt      beta  \
364  0.006946  716  clu_2538 NaN     13110   A  0.452185   
387  0.004229  716  clu_2543 NaN     13110   A -0.464828   
388  0.007822  716  clu_2543 NaN     13110   A  0.435086   
607  0.007159  716  clu_2538 NaN     13273   C -0.271604   

           molecular_trait_id         variant  median_tpm  \
364  1:844498:847654:clu_2538  chr1_13110_G_A         NaN   
387  1:954082:955923:clu_2543  chr1_13110_G_A         NaN   
388  1:954523:955923:clu_2543  chr1_13110_G_A         NaN   
607  1:829104:851927:clu_2538  chr1_13273_G_C         NaN   

    molecular_trait_object_id       maf chromosome   ac type ref  
364           ENSG00000228794  0.053073          1   38  SNP   G  
387           ENSG00000188976  0.053073          1   38  SNP   G  
388           ENSG00000188976  0.053073          1   38  SNP   G  
607           ENSG00000228794  0.156425          1  112  SNP   G  
